# Import packages and get API key

In [34]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key= OPENAI_API_KEY)

# OpenAI interface to making queries

In [35]:
prompt_input = """Write a short message to remind users to be vigilant about 
phishing attacks."""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt_input}
  ],
  temperature= 0.7,
  max_tokens= 400  
)
 
print(response)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-BDBLBKVn5b9gqdPZRjhtf5yzdSlg3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Subject: Stay Vigilant Against Phishing Attacks\n\nDear Team,\n\nWe want to remind everyone to stay vigilant against phishing attacks. Always be cautious when opening emails or messages from unknown senders, and never click on suspicious links or provide personal information without verifying the source. \n\nIf you encounter any suspicious activity, please report it immediately. Your awareness is our best defense!\n\nStay safe,\n[Your Name/Your Team]', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1742481557, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_b8bc95a0ac', usage=CompletionUsage(completion_tokens=85, prompt_tokens=33, total_tokens=118, completion_tokens_details=CompletionTokensDetails(accepted_predi

# LangChain interface for the same query

In [36]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    openai_api_key = OPENAI_API_KEY,
    model_name = "gpt-4o-mini"
)

In [37]:
prompt_input = """Write a coincise message to remind users to be vigilant about phishing attacks."""
 
response = llm.invoke(prompt_input)
print(response.content)

🚨 Important Reminder: Stay Vigilant Against Phishing Scams! 🚨 

Always verify the sender's identity before clicking links or providing personal information. Look out for suspicious emails or messages, even if they seem legitimate. Protect your information—think before you click! 

Stay safe! 🛡️
🚨 Important Reminder: Stay Vigilant Against Phishing Scams! 🚨 

Always verify the sender's identity before clicking links or providing personal information. Look out for suspicious emails or messages, even if they seem legitimate. Protect your information—think before you click! 

Stay safe! 🛡️


# Creating prompt templates (manually)

In [38]:
def generate_text_summary_prompt(text, num_words, tone):
    return f'You are an experienced copywriter. Write a {num_words} words summary the the following text, using a {tone} tone: {text}'

In [39]:
segovia_aqueduct_text = "The Aqueduct of Segovia (Spanish: Acueducto de Segovia) is a Roman aqueduct in Segovia, Spain. It was built around the first century AD to channel water from springs in the mountains 17 kilometres (11 mi) away to the city's fountains, public baths and private houses, and was in use until 1973. Its elevated section, with its complete arcade of 167 arches, is one of the best-preserved Roman aqueduct bridges and the foremost symbol of Segovia, as evidenced by its presence on the city's coat of arms. The Old Town of Segovia and the aqueduct, were declared a UNESCO World Heritage Site in 1985. As the aqueduct lacks a legible inscription (one was apparently located in the structure's attic, or top portion[citation needed]), the date of construction cannot be definitively determined. The general date of the Aqueduct's construction was long a mystery, although it was thought to have been during the 1st century AD, during the reigns of the Emperors Domitian, Nerva, and Trajan. At the end of the 20th century, Géza Alföldy deciphered the text on the dedication plaque by studying the anchors that held the now missing bronze letters in place. He determined that Emperor Domitian (AD 81–96) ordered its construction[1] and the year 98 AD was proposed as the most likely date of completion.[2] However, in 2016 archeological evidence was published which points to a slightly later date, after 112 AD, during the government of Trajan or in the beginning of the government of emperor Hadrian, from 117 AD."
 
input_prompt = generate_text_summary_prompt(text=segovia_aqueduct_text, num_words=20, tone="knowledgeable and engaging")
 
response = llm.invoke(input_prompt)
print(response.content) 

The Aqueduct of Segovia, a UNESCO World Heritage Site, showcases Roman engineering brilliance and transported water to the city for centuries.
The Aqueduct of Segovia, a UNESCO World Heritage Site, showcases Roman engineering brilliance and transported water to the city for centuries.


# Creating prompt templates (with LangChain)

In [40]:
from langchain_core.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template(
    """You are an experienced copywriter. Write a {num_words} words summary the 
    the following text, using a {tone} tone: {text}"""
)

prompt = prompt_template.format(
    text=segovia_aqueduct_text, 
    num_words=20, 
    tone="knowledgeable and engaging"
)

response = llm.invoke(prompt)
print(response.content)

The Aqueduct of Segovia, a Roman engineering marvel completed around 112 AD, remains a UNESCO World Heritage Site and iconic symbol.
The Aqueduct of Segovia, a Roman engineering marvel completed around 112 AD, remains a UNESCO World Heritage Site and iconic symbol.


# Few-shot training

In [41]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate

examples = [
  {
      "number": 6,
      "reasoning": "not divisible by 5 nor by 7",
      "result": "None"
  },
  {
      "number": 15,
      "reasoning": "divisible by 5 but not by 7",
      "result": "Abra"
  },
  {
      "number": 12,
      "reasoning": "not divisible by 5 nor by 7",
      "result": "None"
  },
  {
      "number": 21,
      "reasoning": "divisible by 7 but not by 5",
      "result": "Kadabra"
  },
  {
      "number": 70,
      "reasoning": "divisible by 5 and by 7",
      "result": "Abra Kadabra"
  } ]

example_prompt = PromptTemplate(
    input_variables = ["number", "reasoning", "result"],
    template="{number} \\ {reasoning} \\ {result}"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Classify the following numbers as Abra, Kadabra or Abra Kadabra: {comma_delimited_input_numbers}",
    input_variables=["comma_delimited_input_numbers"]
)

prompt_input = few_shot_prompt.format(
    comma_delimited_input_numbers="3, 4, 5, 7, 8, 10, 11, 13, 35."
)

response = llm.invoke(prompt_input)
print(response.content)

To classify the numbers into "Abra," "Kadabra," and "Abra Kadabra," we can use the given rules based on divisibility:

- "Abra" means divisible by 5 but not by 7.
- "Kadabra" means divisible by 7 but not by 5.
- "Abra Kadabra" means divisible by both 5 and 7.
- None means not divisible by either 5 or 7.

Now let's classify the given numbers (3, 4, 5, 7, 8, 10, 11, 13, 35):

1. **3**: not divisible by 5 nor by 7 - **None**
2. **4**: not divisible by 5 nor by 7 - **None**
3. **5**: divisible by 5 but not by 7 - **Abra**
4. **7**: divisible by 7 but not by 5 - **Kadabra**
5. **8**: not divisible by 5 nor by 7 - **None**
6. **10**: divisible by 5 but not by 7 - **Abra**
7. **11**: not divisible by 5 nor by 7 - **None**
8. **13**: not divisible by 5 nor by 7 - **None**
9. **35**: divisible by both 5 and 7 - **Abra Kadabra**

Summary of classifications:
- 3: None
- 4: None
- 5: Abra
- 7: Kadabra
- 8: None
- 10: Abra
- 11: None
- 13: None
- 35: Abra Kadabra
To classify the numbers into "Abra,